In [ ]:
import os
import Cell_BLAST as cb
import utils
os.environ["CUDA_VISIBLE_DEVICES"] = utils.pick_gpu_lowest_memory()
cb.config.RANDOM_SEED = 0
cb.config.N_JOBS = 4
fixed_model_kwargs = dict(
    latent_dim=10, cat_dim=20,
    epoch=100, patience=10  # only for this dataset
)

In [ ]:
cb.__version__

---
## Cao_2019

In [ ]:
cao_2019 = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Cao_2019/data.h5")
utils.peek(cao_2019, "build/mouse_embryo/Cao_2019")
cao_2019.obs.head()

In [ ]:
cao_2019.obs["donor"] = "donor_" + cao_2019.obs["donor"].astype(int).astype(str)
cao_2019.obs["lifestage"] = "embryo_" + cao_2019.obs["lifestage"].astype(str)
cao_2019.obs.dtypes

In [ ]:
opt_model_kwargs = dict(prob_module_kwargs=dict(lambda_reg=0.01))
cao_2019_model = cb.directi.fit_DIRECTi(
    cao_2019, cao_2019.uns["scanpy_genes"],
    **fixed_model_kwargs, **opt_model_kwargs
)
cao_2019.latent = cao_2019_model.inference(cao_2019)

In [ ]:
ax = cao_2019.visualize_latent(
    "cell_type1", method="UMAP", size=0.1,
    dr_kws=dict(n_neighbors=7, min_dist=0.7),
    scatter_kws=dict(rasterized=True)
)
ax.get_figure().savefig("build/mouse_embryo/Cao_2019/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = cao_2019.visualize_latent(
    "cell_ontology_class", method="UMAP", size=0.1,
    dr_kws=dict(n_neighbors=7, min_dist=0.7),
    scatter_kws=dict(rasterized=True)
)
ax.get_figure().savefig("build/mouse_embryo/Cao_2019/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = cao_2019.visualize_latent(
    "lifestage", method="UMAP", size=0.1,
    dr_kws=dict(n_neighbors=7, min_dist=0.7),
    scatter_kws=dict(rasterized=True)
)
ax.get_figure().savefig("build/mouse_embryo/Cao_2019/lifestage.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
cao_2019.write_dataset("build/mouse_embryo/Cao_2019/Cao_2019.h5")

In [ ]:
%%capture capio
cao_2019_models = [cao_2019_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    cao_2019_models.append(cb.directi.fit_DIRECTi(
        cao_2019, cao_2019.uns["scanpy_genes"],
        **fixed_model_kwargs, **opt_model_kwargs,
        random_seed=i
    ))
cao_2019_blast = cb.blast.BLAST(cao_2019_models, cao_2019)
cao_2019_blast.save("build/mouse_embryo/Cao_2019")

In [ ]:
with open("build/mouse_embryo/Cao_2019/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/mouse_embryo/Cao_2019/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(cao_2019_blast, "build/mouse_embryo/Cao_2019")

In [ ]:
%%writefile "build/mouse_embryo/Cao_2019/predictable.txt"
cell_ontology_class
cell_type1
ligestage